In [164]:
from sympy import Matrix, frac

A = Matrix([
    [2,  3, 1, 0,  0],
    [3, -5, 0, 1,  0],
    [5,  3, 0, 0, -1],
])

b = Matrix([19, 17, 17])

c = Matrix([-5, -1, 0, 0, 0])

In [165]:
def simplex_method(A, b, c, basis):
    global deltas
    #print('step')
    #print(A)
    #print(b)
    #print(basis)
    m, n = A.shape

    # compute delta_k
    deltas = [
        c[j] - sum(c[basis[i]]*A[i, j] for i in range(m))
        for j in range(n)
    ]

    #print('deltas', deltas)

    # check if optimal vertex is reached
    if all(delta >= 0 for delta in deltas):
        xstar = [0] * n
        for i in range(m):
            xstar[basis[i]] = b[i]
        return xstar

    k = deltas.index(min(deltas))  # determine in_index

    # check if target function is unbounded
    if all(A[i, k] <= 0 for i in range(m)): return

    # compute theta_k
    thetas = [
        b[i]/A[i, k] if A[i, k] > 0 else float('inf')
        for i in range(m)
    ]

    #print('thetas', thetas)

    l = thetas.index(min(thetas))  # determine out_index
    basis[l] = k                   # change basis

    # perform gauss transformations
    b[l] /= A[l, k]
    A[l, :] /= A[l, k]
    for i in range(m):
        if i == l: continue
        b[i] -= b[l] * A[i, k]
        A[i, :] -= A[l, :] * A[i, k]

    return simplex_method(A, b, c, basis)   # start over

In [166]:
# M-method
m, n = A.shape
M = max(max(abs(A)), max(abs(b)), max(abs(c))) + 1
basis = []
ys = 0

for i in range(m):
    if A[i, n-m+i] == 1:
        basis.append(n-m+i)
    else:
        col = [0] * m
        col[i] = 1
        A = A.row_join(Matrix(col))
        c = c.col_join(Matrix([M]))
        basis.append(n + ys)
        ys += 1

xstar = simplex_method(A, b, c, basis)

if xstar is None or any(x > 0 for x in xstar[n:]):
    print("No solution")

print('x* =', xstar[:n])
print('L* =', -c.dot(xstar))

x* = [146/19, 23/19, 0, 0, 476/19]
L* = 753/19
Matrix([[23/19], [476/19], [146/19]])


In [167]:
def gomory_cut(A, b, x, basis):
    if all(type(beta) is int or beta.is_integer for beta in x): return x
    
    global deltas
    l = -1
    for i, beta_i in enumerate(x):
        if not beta_i.is_integer:
            l = i
            break
            
    index = basis.index(l)
    
    new_row = [-frac(alpha) for alpha in A[index, :]]
    new_row.append(1)
    
    b = b.col_join(Matrix([-frac(x[l])]))
    A = A.row_join(Matrix([0]*m))
    A = A.col_join(Matrix([new_row]))

    deltas = [-delta/alpha if alpha < 0 else float('inf') for delta, alpha in zip(deltas, new_row)]
    k = deltas.index(min(deltas)) # in_index
    
    basis.append(k)
    l = m # out_index

    b[l] /= A[l, k]
    A[l, :] /= A[l, k]
    for i in range(m):
        if i == l: continue
        b[i] -= b[l] * A[i, k]
        A[i, :] -= A[l, :] * A[i, k]

    x = simplex_method(A, b, c, basis)
    
    return gomory_cut(A[:m, :n], Matrix(b[:m]), x[:n], basis[:m])

In [177]:
c = Matrix(c[:n])
xstar = gomory_cut(A[:, :n], b, xstar[:n], basis)
print('x* =', xstar)
print('L* =', -c.dot(xstar))

x* = [7, 1, 2, 0, 21]
L* = 36
